In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
folder_path = "/kaggle/input/raw-instinct"
data_path = os.listdir(folder_path)
raw = os.path.join(folder_path, [r for r in data_path if "RAW" in r][-1])

rice = [os.path.join(raw, rice, "train")for rice in os.listdir(raw)]

In [3]:
def get_data(path, test_train):
    rice = [os.path.join(path, rice, test_train)for rice in os.listdir(path)]

    data_list = []
    for r in rice:
        files_in_r = os.listdir(r)
        y = r.split("/")[-2]
        
        for file_name in files_in_r:
            
            file_path = os.path.join(r, file_name)
            x = np.load(file_path)
            data_list.append((y, x))

    df = pd.DataFrame(data_list, columns=['Y', 'X'])
    return df

train_df = get_data(raw, "train")
test_df = get_data(raw, "test")
val_df = get_data(raw, "validation")

In [4]:
X_train = np.array(train_df['X'].tolist()).astype('float32') / 255.0
y_train = np.array(pd.get_dummies(train_df['Y']))

X_test = np.array(test_df['X'].tolist()).astype('float32') / 255.0
y_test = np.array(pd.get_dummies(test_df['Y']))

X_val = np.array(val_df['X'].tolist()).astype('float32') / 255.0
y_val = np.array(pd.get_dummies(val_df['Y']))

if len(X_train.shape) == 3:
    X_train = np.expand_dims(X_train, axis=-1)
    X_test = np.expand_dims(X_test, axis=-1)
    X_val = np.expand_dims(X_val, axis=-1)

#print("Train Images Shape, type and first input:", X_train.shape, type(X_train), X_train[0][0])
#print("Train Labels Shape, type and first input:", y_train.shape, type(y_train), y_train[0])
#print("Test Images Shape, type and first input:", X_test.shape, type(X_test), X_test[0][0])
#print("Test Labels Shape, type and first input:", y_test.shape, type(y_test), y_test[0])
#print("Validation Images Shape, type and first input:", X_val.shape, type(X_val), X_val[0][0])
#print("Validation Labels Shape, type and first input:", y_val.shape, type(y_val), y_val[0][0])

In [5]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2024-02-17 11:46:54.314849: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-17 11:46:54.314952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-17 11:46:54.619619: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2 Physical GPUs, 2 Logical GPUs


In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Add, Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import keras_tuner as kt

In [7]:
def residual_block(x, filters, conv_num=3, stride=1):
    # Shortcut
    shortcut = x

    for i in range(conv_num):
        x = Conv2D(filters, (3, 3), strides=stride if i == 0 else 1, padding="same")(x)
        x = BatchNormalization()(x)

        if i < conv_num - 1:
            x = ReLU()(x)

    # If the number of filters is not the same, apply a linear projection (1x1 conv)
    if stride != 1 or x.shape[-1] != shortcut.shape[-1]:
        shortcut = Conv2D(filters, (1, 1), strides=stride, padding="same")(shortcut)

    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

def resnet_34(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    x = Conv2D(64, (7, 7), strides=2, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = residual_block(x, 64, conv_num=2, stride=2)
    x = residual_block(x, 128, conv_num=2, stride=2)
    x = residual_block(x, 256, conv_num=2, stride=2)
    x = residual_block(x, 512, conv_num=2, stride=2)

    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [8]:
model = resnet_34((40, 40, 1), 5)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#history = model.fit(X_train, y_train, epochs=10, batch_size=32)

In [9]:
def build_model(hp):
    # Hyperparameters to tune
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    num_units = hp.Int('num_units', min_value=32, max_value=512, step=32)

    # Building ResNet-34 model
    model = resnet_34((40, 40, 1), 5)  # Adjust the input shape and number of units according to your dataset

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [10]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='resnet34_tuning'
)

In [11]:
from tqdm.keras import TqdmCallback
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

In [12]:
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[stop_early])

Trial 10 Complete [00h 00m 52s]
val_accuracy: 0.9519860148429871

Best val_accuracy So Far: 0.9685726761817932
Total elapsed time: 00h 16m 41s


In [13]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [16]:
for hp, value in best_hps.values.items():
    print(f"{hp}: {value}")

learning_rate: 0.00018342932760963753
num_units: 352
